# Filter and transform raw data

**_Objective:_** Take the raw data and perform some filtering and transformations to get it into a more usuable format for later work.

## Required libraries

In [1]:
from datetime import datetime
import os
import pandas as pd

## Required function

In [2]:
def zodiac_sign(day, month):
    # Adapted from:
    #     https://stackoverflow.com/questions/61537916/map-column-birthdates-in-python-pandas-df-to-astrology-signs
    #
    # PURPOSE
    #     This function will receive a day and month as arguments and 
    #     then return the zodiac sign for the date.
    #
    # INPUTS
    #     day   := The day of the month. Type should be numeric.
    #     month := The full name of the month. Type should be a string and all 
    #              chars should be lowercase.
    #
    # OUTPUT
    #     N/A.
    #
    # RETURNS
    #     The zodiac sign of the date provided. Type will be a string.
    if month == 'december': 
        return 'Sagittarius' if (day < 22) else 'Capricorn'
    elif month == 'january': 
        return 'Capricorn' if (day < 20) else 'Aquarius'
    elif month == 'february': 
        return 'Aquarius' if (day < 19) else 'Pisces'
    elif month == 'march': 
        return 'Pisces' if (day < 21) else 'Aries'
    elif month == 'april': 
        return 'Aries' if (day < 20) else 'Taurus'
    elif month == 'may': 
        return 'Taurus' if (day < 21) else 'Gemini'
    elif month == 'june': 
        return 'Gemini' if (day < 21) else 'Cancer'
    elif month == 'july': 
        return 'Cancer' if (day < 23) else 'Leo'
    elif month == 'august': 
        return 'Leo' if (day < 23) else 'Virgo'
    elif month == 'september': 
        return 'Virgo' if (day < 23) else 'Libra'
    elif month == 'october': 
        return 'Libra' if (day < 23) else 'Scorpio'
    elif month == 'november': 
        return 'Scorpio' if (day < 22) else 'Sagittarius'

## Bring the data in

In [3]:
os.chdir("../data/raw/")

In [4]:
dat = pd.read_csv("nuforc_reports.csv")

In [5]:
dat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141261 entries, 0 to 141260
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   summary         141189 non-null  object 
 1   country         140944 non-null  object 
 2   city            140783 non-null  object 
 3   state           131681 non-null  object 
 4   date_time       138601 non-null  object 
 5   shape           134962 non-null  object 
 6   duration        133645 non-null  object 
 7   stats           141261 non-null  object 
 8   report_link     141261 non-null  object 
 9   text            141227 non-null  object 
 10  posted          138601 non-null  object 
 11  city_latitude   115440 non-null  float64
 12  city_longitude  115440 non-null  float64
dtypes: float64(2), object(11)
memory usage: 14.0+ MB


## Remove features that will not be needed

In [6]:
dat = dat.drop(["summary",
                "city",
                "stats",
                "report_link",
                "posted",
                "duration"], axis=1)

## Filter to only CONUS locations

In [7]:
dat = dat[dat["country"] == "USA"]
dat = dat.drop(["country"], axis=1)

In [8]:
# While technically not a state, we will keep Washington, D.C. as DC
state_list = ["AL", "AZ", "AR", "CA", "CO", "CT", "DE", "DC", "FL", "GA", "ID",
              "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", "MA", "MI", "MN", 
              "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", 
              "OH", "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VT",
              "VA", "WA", "WV", "WI", "WY"]

In [9]:
dat = dat[dat["state"].isin(state_list)]
del state_list

## Get rid of sightings that do not have a lattitude or longitude

In [10]:
dat = dat[dat["city_latitude"].notna()]

In [11]:
dat = dat[dat["city_longitude"].notna()]

## Fix the `shape` field

In [12]:
dat["shape"].unique()

array([nan, 'light', 'circle', 'sphere', 'unknown', 'chevron', 'cigar',
       'other', 'triangle', 'oval', 'teardrop', 'flash', 'rectangle',
       'formation', 'disk', 'fireball', 'egg', 'changing', 'diamond',
       'cone', 'cylinder', 'cross', 'delta', 'star'], dtype=object)

From the above output we see that there are some missing values (`nan`) for the shape of the UFO and also a recording for `unknown` shapes. Here we will change those `nan`s to `unknown` and also title case all the shape recordings.

In [13]:
shapes = []
for shape in dat["shape"]:
    if type(shape) == float:
        shapes.append("Unknown")
    else:
        shapes.append(shape.title())
dat["shape"] = shapes
del shape
del shapes

## Get rid of sightings that do not have a date recorded

In [14]:
dat = dat[dat["date_time"].notna()]

## Git rid of sightings that do not have any text describing the sighting

In [15]:
dat = dat[dat["text"].notna()]

## Extract the date, day of week, and weekend status from `date_time`

In [16]:
dat["date"] = pd.to_datetime(dat["date_time"]).dt.date

In [17]:
dat["day_of_week"] = pd.to_datetime(list(dat["date_time"])).day_name()

In [18]:
dat["weekend_flag"] = ["Yes" if day in ["Saturday", "Sunday"] else "No" 
                       for day in dat["day_of_week"]]

## Get the zodiac sign of the sighting date

In [19]:
dat["temp_date"] = pd.to_datetime(dat["date"])
dat["zodiac_sign_of_sighting_date"] = dat["temp_date"].apply(lambda x: zodiac_sign(x.day, x.strftime("%B").lower()))
dat = dat.drop(["temp_date"], axis=1)

## Write the data frame to file

In [20]:
os.chdir("../interim/")

In [21]:
file_name_root = "interim_no_county_raw_ufo_data_"
date_time_str = datetime.today().strftime("%Y-%m-%d %H:%M")
date_str = date_time_str.split()[0].replace("-", "") + "_"
time_str = date_time_str.split()[1].replace(":", "")
file_ext = '.csv'
del date_time_str
file_name = file_name_root + date_str + time_str + file_ext
del file_name_root
del date_str
del time_str
del file_ext


In [22]:
print(file_name)

interim_no_county_raw_ufo_data_20230429_2035.csv


In [23]:
dat.to_csv(file_name, sep=",", index=False)
del file_name

In [24]:
dat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 107980 entries, 0 to 141260
Data columns (total 10 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   state                         107980 non-null  object 
 1   date_time                     107980 non-null  object 
 2   shape                         107980 non-null  object 
 3   text                          107980 non-null  object 
 4   city_latitude                 107980 non-null  float64
 5   city_longitude                107980 non-null  float64
 6   date                          107980 non-null  object 
 7   day_of_week                   107980 non-null  object 
 8   weekend_flag                  107980 non-null  object 
 9   zodiac_sign_of_sighting_date  107980 non-null  object 
dtypes: float64(2), object(8)
memory usage: 9.1+ MB
